<a href="https://colab.research.google.com/github/cardazuluaga/DLCourse/blob/main/Ej8TweetEmotionRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nlp

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random


def show_history(h):
    epochs_trained = len(h.history['loss'])
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(0, epochs_trained), h.history.get('accuracy'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_accuracy'), label='Validation')
    plt.ylim([0., 1.])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(0, epochs_trained), h.history.get('loss'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_loss'), label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def show_confusion_matrix(y_true, y_pred, classes):
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y_true, y_pred, normalize='true')

    plt.figure(figsize=(8, 8))
    sp = plt.subplot(1, 1, 1)
    ctx = sp.matshow(cm)
    plt.xticks(list(range(0, 6)), labels=classes)
    plt.yticks(list(range(0, 6)), labels=classes)
    plt.colorbar(ctx)
    plt.show()


print('Using TensorFlow version', tf.__version__)

In [ ]:
#Se importan los datos

!pip install -U datasets

from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion") #https://huggingface.co/datasets/dair-ai/emotion

In [ ]:
#Se separa los datos

train = dataset['train']
val = dataset['validation']
test = dataset['test']

In [ ]:
#Se define una función para obtener las características
#(entradas) y las etiquetas (salidas)

def get_tweet(data):
  tweets = [x['text'] for x in data]
  labels = [x['label'] for x in data]
  return tweets, labels

In [ ]:
tweets, labels = get_tweet(train)
labels = np.array(labels)

In [ ]:
tweets[1], labels[1]



Los labels son:

0 para sadness

1 para joy

2 para love

3 para anger

4 para fear

5 para surprise

In [ ]:
#Se carga la librería para transformar texto en secuencias

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='UNK')
tokenizer.fit_on_texts(tweets)

In [ ]:
tokenizer.texts_to_sequences([tweets[0]])

In [ ]:
tweets[0]

In [ ]:
# Padding y truncamiento

lengths = [len(t.split(' ')) for t in tweets]
plt.hist(lengths, bins = len(set(lengths)))
plt.show()

In [ ]:
maxlen = 50

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def get_sequences(tokenizer,tweets):
  sequences = tokenizer.texts_to_sequences(tweets)
  padded = pad_sequences(sequences, truncating='post',padding='post', maxlen=maxlen)
  return padded

padded_train_seq = get_sequences(tokenizer,tweets)
padded_train_seq = np.array(padded_train_seq)

In [ ]:
padded_train_seq[0]

In [ ]:
plt.hist(labels,bins=11)
plt.show()

In [ ]:
#Se define el modelo

model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(10000,16,input_length=maxlen),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
        tf.keras.layers.Dense(6,activation='softmax')
])

model.compile(

    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

In [ ]:
#Se entrena el modelo

val_tweets, val_labels = get_tweet(val)
val_labels = np.array(val_labels)
val_seq = get_sequences(tokenizer,val_tweets)
val_seq = np.array(val_seq)

h = model.fit(
    padded_train_seq, labels,
    validation_data=(val_seq, val_labels),
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2)
    ]
)

In [ ]:
#Se evalua el modelo

show_history(h)

In [ ]:
test_tweets, test_labels = get_tweet(test)
test_labels = np.array(test_labels)
test_seq = get_sequences(tokenizer,test_tweets)
test_seq = np.array(test_seq)

In [ ]:
i = random.randint(0,len(test_labels) - 1)

print('Tweet:', test_tweets[i])
print('Emocion:', test_labels[i])

p = model.predict(np.expand_dims(test_seq[i],axis=0))[0]
pred_class = np.argmax(p).astype('uint8')

print('Emocion predicha', pred_class)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

p = model.predict(test_seq)
classes_pred=np.argmax(p,axis=1)

cm = confusion_matrix(test_labels, classes_pred)
cm_display = ConfusionMatrixDisplay(cm).plot()